In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
import io
import base64
from prettytable import PrettyTable

In [11]:
def determine_consequent(rule):
    negative_count = 0
    if rule['Kualitas Tidur'] == 'Buruk':
        negative_count += 1
    if rule['Performa Akademik'] in ['Rendah', 'Sedang']:
        negative_count += 1
    if rule['Hubungan Dosen-Mhs'] == 'Buruk':
        negative_count += 1
    if rule['Support Sosial'] == 'Buruk':
        negative_count += 1
    if rule['Kondisi Kehidupan'] == 'Buruk':
        negative_count += 1

    if negative_count >= 4:
        return 'Tinggi'
    elif negative_count >= 2:
        return 'Sedang'
    else:
        return 'Normal'

In [12]:
def two_category_membership(x, low=30, high=70):
    if x <= low:
        return {'Buruk': 1, 'Baik': 0}
    elif x < high:
        return {'Buruk': (high - x)/(high - low), 'Baik': (x - low)/(high - low)}
    else:
        return {'Buruk': 0, 'Baik': 1}

def academic_performance_membership(x, low=25, mid=50, high=75):
    if x <= low:
        return {'Rendah': 1, 'Sedang': 0, 'Tinggi': 0}
    elif x < mid:
        rendah = (mid - x) / (mid - low)
        sedang = (x - low) / (mid - low)
        return {'Rendah': rendah, 'Sedang': sedang, 'Tinggi': 0}
    elif x < high:
        sedang = (high - x) / (high - mid)
        tinggi = (x - mid) / (high - mid)
        return {'Rendah': 0, 'Sedang': sedang, 'Tinggi': tinggi}
    else:
        return {'Rendah': 0, 'Sedang': 0, 'Tinggi': 1}


In [13]:
def compute_stress_level(sleep_quality, academic_performance, hubungan, social_support, living_condition):
    input_dict = {
        'Kualitas Tidur': sleep_quality,
        'Performa Akademik': academic_performance,
        'Hubungan Dosen-Mhs': hubungan,
        'Support Sosial': social_support,
        'Kondisi Kehidupan': living_condition
    }
    
    # Hitung nilai keanggotaan
    memberships = {}
    for factor, value in input_dict.items():
        if factor == 'Performa Akademik':
            memberships[factor] = academic_performance_membership(value)
        else:
            memberships[factor] = two_category_membership(value)
    
    factor_names = list(input_dict.keys())
    category_lists = [list(memberships[factor].keys()) for factor in factor_names]
    
    total_alpha_z = 0
    total_alpha = 0
    for combo in itertools.product(*category_lists):
        mem_values = [memberships[factor_names[i]][combo[i]] for i in range(5)]
        alpha = min(mem_values)
        if alpha > 0:
            rule = {factor_names[i]: combo[i] for i in range(5)}
            consequent = determine_consequent(rule)
            # Z nilai output fuzzy (misal: Normal=25, Sedang=50, Tinggi=75, adjusted dengan alpha)
            if consequent == 'Tinggi':
                z_i = 75 * alpha
            elif consequent == 'Sedang':
                z_i = 50 * alpha
            else:
                z_i = 25 * alpha
            total_alpha_z += alpha * z_i
            total_alpha += alpha
    
    return total_alpha_z / total_alpha if total_alpha > 0 else 50

In [14]:
kualitas_tidur = int(input('Kualitas Tidur : '))
performa_akademik = int(input('Performa Akademik : '))
hub_mhs_dosen = int(input('Hubungan Mahasiswa dengan Dosen : '))
support_sosial = int(input('Support Sosial : '))
kondisi_kehidupan = int(input('Kondisi Kehidupan : '))

print('>>> Hasil Deteksi <<<')
stress_level = compute_stress_level(kualitas_tidur, performa_akademik, hub_mhs_dosen, support_sosial, kondisi_kehidupan)
if stress_level < 35:
    tingkat_stress = "Normal"
elif stress_level < 65:
    tingkat_stress = "Sedang"
else:
    tingkat_stress = "Tinggi"

print(f"Level Stress    : {stress_level}")
print(f"Tingkat Stress  : {tingkat_stress}")

>>> Hasil Deteksi <<<
Level Stress    : 25.0
Tingkat Stress  : Normal


In [ ]:
def generate_rules():
    sleep_options = ['Buruk', 'Baik']
    academic_options = ['Rendah', 'Sedang', 'Tinggi']
    hubungan_options = ['Buruk', 'Baik']
    support_options = ['Buruk', 'Baik']
    hidup_options = ['Buruk', 'Baik']

    rules = []
    for idx, combo in enumerate(itertools.product(sleep_options, academic_options, hubungan_options, support_options, hidup_options), 1):
        sleep, academic, hubungan, support, hidup = combo
        negative_count = 0
        if sleep == 'Buruk':
            negative_count += 1
        if academic in ['Rendah', 'Sedang']:  # 'Tinggi' dianggap positif
            negative_count += 1
        if hubungan == 'Buruk':
            negative_count += 1
        if support == 'Buruk':
            negative_count += 1
        if hidup == 'Buruk':
            negative_count += 1

        if negative_count >= 4:
            output = 'Tinggi'
        elif negative_count >= 2:
            output = 'Sedang'
        else:
            output = 'Normal'

        rules.append([idx, sleep, academic, hubungan, support, hidup, output])
    
    return rules

def show_tabel():
    rules = generate_rules()
    table = PrettyTable()
    table.field_names = ["No", "Kualitas Tidur", "Performa Akademik", "Hub. Dosen-Mhs", "Support Sosial", "Kondisi Kehidupan", "Output"]
    
    for rule in rules:
        table.add_row(rule)

    print(table)


show_tabel()


+----+----------------+-------------------+----------------+----------------+-------------------+--------+
| No | Kualitas Tidur | Performa Akademik | Hub. Dosen-Mhs | Support Sosial | Kondisi Kehidupan | Output |
+----+----------------+-------------------+----------------+----------------+-------------------+--------+
| 1  |     Buruk      |       Rendah      |     Buruk      |     Buruk      |       Buruk       | Tinggi |
| 2  |     Buruk      |       Rendah      |     Buruk      |     Buruk      |        Baik       | Tinggi |
| 3  |     Buruk      |       Rendah      |     Buruk      |      Baik      |       Buruk       | Tinggi |
| 4  |     Buruk      |       Rendah      |     Buruk      |      Baik      |        Baik       | Sedang |
| 5  |     Buruk      |       Rendah      |      Baik      |     Buruk      |       Buruk       | Tinggi |
| 6  |     Buruk      |       Rendah      |      Baik      |     Buruk      |        Baik       | Sedang |
| 7  |     Buruk      |       Rendah 

In [16]:
import itertools
import csv

def generate_rules():
    sleep_options = ['Buruk', 'Baik']
    academic_options = ['Rendah', 'Sedang', 'Tinggi']
    hubungan_options = ['Buruk', 'Baik']
    support_options = ['Buruk', 'Baik']
    hidup_options = ['Buruk', 'Baik']

    rules = []
    for idx, combo in enumerate(itertools.product(sleep_options, academic_options, hubungan_options, support_options, hidup_options), 1):
        sleep, academic, hubungan, support, hidup = combo
        negative_count = 0
        if sleep == 'Buruk':
            negative_count += 1
        if academic in ['Rendah', 'Sedang']:  # 'Tinggi' dianggap positif
            negative_count += 1
        if hubungan == 'Buruk':
            negative_count += 1
        if support == 'Buruk':
            negative_count += 1
        if hidup == 'Buruk':
            negative_count += 1

        if negative_count >= 4:
            output = 'Tinggi'
        elif negative_count >= 2:
            output = 'Sedang'
        else:
            output = 'Normal'

        rules.append([
            idx, sleep, academic, hubungan, support, hidup, output
        ])
    
    return rules

def save_rules_to_csv(filename="rules_stress_fuzzy.csv"):
    rules = generate_rules()
    headers = ["No", "Kualitas Tidur", "Performa Akademik", "Hubungan Dosen-Mhs", "Support Sosial", "Kondisi Kehidupan", "Output"]

    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        writer.writerows(rules)

    print(f"✅ {len(rules)} rules berhasil disimpan ke {filename}")

# Jalankan fungsi
save_rules_to_csv()


✅ 48 rules berhasil disimpan ke rules_stress_fuzzy.csv
